In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
import plotly.express as px
import plotly.io as pio
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
# plt.rcParams['figure.figsize'] = [40, 7]
jtplot.style(theme="monokai", context="notebook", ticks=True,  figsize=(40, 10))

In [4]:
jtplot.style(theme="monokai", context="notebook", ticks=True,  figsize=(40, 10))

In [5]:
TOP_10_CAPITALIZATION = ['btcusd', 'ethusd', 'eosusd', 'ltcusd', 'xrpusd', 'babusd', 'xmrusd', 'neousd', 'iotusd', "dshusd"]
timestamp2datetime = lambda x: datetime.fromtimestamp(int(x) / 1000.0)

In [ ]:
# try out with btc
btc_df = pd.read_csv(
        f"data/original/btcusd.csv",
        sep=',',
        parse_dates=['time'],
        index_col='time',
        date_parser=timestamp2datetime
    )
btc_df["middle"] = (btc_df["open"] + btc_df["close"]) / 2

In [ ]:
btc_1min_df = (
    btc_df
    .groupby(pd.Grouper(freq="1min"))
    .agg(
        open=("open", "first"),
        close=("close", "last"),
        high=("high", np.max),
        low=("low", np.min),
        volume=("volume", np.sum),
        middle_mean=("middle", np.mean),
        middle_median=("middle", np.median),
    )
    .fillna(method='ffill')
)

In [ ]:
btc_1min_df = btc_1min_df.fillna(method='ffill')

In [ ]:
print("Total Bins:", btc_1min_df.shape[0])
print("Bins without Trading:", btc_1min_df[ btc_1min_df["volume"] == 0 ].shape[0])
btc_1min_df.head()

In [ ]:
daily_missing_bins_df.head()

# Create minute-binned data

In [ ]:
top10_1min_df = pd.DataFrame()
for pair in TOP_10_CAPITALIZATION:
    print(pair)
    min1_df = pd.read_csv(
        f"data/original/{pair}.csv",
        sep=',',
        parse_dates=['time'],
        index_col='time',
        date_parser=timestamp2datetime
    )
    min1_df["middle"] = (min1_df["open"] + min1_df["close"]) / 2
    min1_df = (
        min1_df
        .groupby(pd.Grouper(freq="1min"))
        .agg(
            open=("open", "first"),
            close=("close", "last"),
            high=("high", np.max),
            low=("low", np.min),
            volume=("volume", np.sum),
            middle_median=("middle", np.median),
        )
        .fillna(method='ffill')
    )
    min1_df["pair"] = pair
    top10_1min_df = pd.concat( [top10_1min_df, min1_df] ) 

In [ ]:
# top10_1min_df.to_csv(
#     "data/1min/top10_merged.csv.gz",
#     sep=",",
#     chunksize=100000,
#     compression='gzip',
# )

In [ ]:
top10_1min_df.to_hdf("data/1min/top10_merged.h5", mode="w")

In [ ]:
top10_1min_df.head()

In [ ]:
top10_1min_df = top10_1min_df.reset_index()
top10_1min_df = top10_1min_df.set_index([ "pair", "time"  ])

In [ ]:
top10_1min_df.head()

# Investigate missing bins

In [ ]:
daily_missing_bins_df = (
    top10_1min_df[ top10_1min_df["volume"] == 0 ]
    .groupby(
        [ pd.Grouper(level="pair"), pd.Grouper(level="time", freq="1D") ]
    ).agg( daily_missing_bins=("volume", "count") )
    .reset_index()
)

In [ ]:
fig = px.line(daily_missing_bins_df[ daily_missing_bins_df["time"] > "2017-12-31" ] ,
              x="time", y="daily_missing_bins", color="pair",
              title="Daily missing 1min-Bins for Top 10 Coins for 2018-19")
# add total minutes for a day as dotted line
fig.add_shape(
    type="line", line_color="red", # line_width=3, opacity=1,
    line_width=3,
    line_dash="dot",
    x0=0, x1=1, 
    xref="paper", 
    y0=24*60, y1=24*60, 
    yref="y"
)
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.show()

In [ ]:
# pio.write_html(fig, file='exploration/1min/daily_missing_1min_bins.html')

# Investigate returns below transaction cost

## Enrich data set with lagged returns and scaled volumes

In [ ]:
deltas = (
    [ 5, 10 ]
  + [ x * 20 for x in range(1, 7) ] 
  + [ 120+ 120 * x for x in range(1, 12) ]
  + [ 1440 + 240 * x for x in range(1, 3) ]
)    
print( [ f"{x}min" for x in deltas ] ) 

In [ ]:
scalers = {}
top10_1min_df["volume_scaled"] = top10_1min_df["volume"]
for pair in TOP_10_CAPITALIZATION:
    print(pair)
    pair_volume_training = (
        top10_1min_df
        [ (top10_1min_df.index.get_level_values("pair") == pair)  & (top10_1min_df["volume"] != 0) & (top10_1min_df.index.get_level_values("time") < "2019-11") & (top10_1min_df.index.get_level_values("time") >= "2019-01-01")]
        [["volume"]]
    )
    quantile_transformer = QuantileTransformer(random_state=0)
    quantile_transformer.fit([ [x] for x in pair_volume_training["volume"].values ])
    scalers[pair] = quantile_transformer
    transformed_data = quantile_transformer.transform([ [x] for x in top10_1min_df["volume"][ (top10_1min_df.index.get_level_values("pair") == pair) ].values ])
    transformed_data = [ x[0] for x in transformed_data ]
    top10_1min_df["volume_scaled"][ (top10_1min_df.index.get_level_values("pair") == pair) ] = transformed_data

In [ ]:
top10_1min_df[ ["volume_scaled", "volume"] ].head()

In [ ]:
top10_1min_df[ ["volume_scaled", "volume"] ].tail()

In [ ]:
top10_1min_df[["volume", "volume_scaled"]][ (40 <= top10_1min_df["volume"]) & (40 >= top10_1min_df["volume"]) & (top10_1min_df.index.get_level_values("pair") == "ethusd") ].head()

In [ ]:
top10_1min_df[["volume", "volume_scaled"]][ (40 <= top10_1min_df["volume"]) & (40 >= top10_1min_df["volume"]) & (top10_1min_df.index.get_level_values("pair") == "btcusd") ].head()

In [ ]:
top10_1min_df[f"middle_return_1min"] = top10_1min_df["middle_median"].groupby(['pair']).pct_change(periods=1)
for delta in deltas:
    print(delta)
    top10_1min_df[f"middle_return_{ int(delta) }min"] = top10_1min_df["middle_median"].groupby(['pair']).pct_change(periods=delta)
    top10_1min_df[f"volume_scaled_{ int(delta) }min"] = top10_1min_df["middle_median"].groupby(['pair']).shift(periods=delta)

In [37]:
def get_3state_price_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    
    if abs(price_return) - transaction_cost > 0:
        if cross_sectional_median_column:
            if price_return > max(row[cross_sectional_median_column], 0):
                return 1
            elif price_return < min(row[cross_sectional_median_column], 0):
                return -1
            else:
                return 0
        else:
            if price_return > 0:
                return 1
            else:
                return -1
    else:
        return 0

def get_2state_up_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    if cross_sectional_median_column:
        if price_return > row[cross_sectional_median_column] and price_return - transaction_cost > 0:
            return 1
        else:
            return -1
    else:
        if price_return - transaction_cost > 0:
            return 1
        else:
            return -1
        
def get_2state_down_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    if cross_sectional_median_column:
        if price_return < row[cross_sectional_median_column] and -price_return - transaction_cost > 0:
            return -1
        else:
            return 1
    else:
        if -x[ return_column ] - transaction_cost > 0:
            return -1
        else:
            return 1           

In [ ]:
for delta in deltas:
    if 120 <= delta <= 600:
        top10_1min_df[f"future_middle_return_{delta}min"] = ( 
            ( top10_1min_df["middle_median"].groupby(['pair']).shift(-(delta + 1)) - top10_1min_df["middle_median"].groupby(['pair']).shift(-1) ) 
            / top10_1min_df["middle_median"].groupby(['pair']).shift(-1)
        )
        top10_1min_df[f"future_2state_movement_{delta}min"] = top10_1min_df[f"future_middle_return_{delta}min"].apply(get_2state_price_movement)
        top10_1min_df[f"future_3state_movement_{delta}min"] = top10_1min_df[f"future_middle_return_{delta}min"].apply(get_3state_price_movement)

In [ ]:
top10_1min_df.head()

In [ ]:
# save to gzip-csv
# (
#     top10_1min_df
#     [ (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]
#     .drop(columns=["open", "close", "high", "low", "volume"])
#     .to_csv(
#         "data/1min/top10_2019_train_test.csv.gz",
#          sep=",",
#          chunksize=1000000,
#          compression='gzip',
#     )
    
# )

In [ ]:
# top10_1min_df[ top10_1min_df["volume_scaled"] != 0 ][ "future_middle_return_360min" ]

top10_1min_df[ [x for x in top10_1min_df.columns if "state" in x ] ].head()

In [4]:
top10_1min_df = pd.read_csv(
    f"../data/1min/top10_2019_train_test.csv.gz",
    sep=',',
    parse_dates=["time"],
    index_col=['time', 'pair'],
    infer_datetime_format=True,
    compression='gzip',
)
top10_1min_df.head()

,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,future_3state_movement_240min,future_middle_return_360min,future_2state_movement_360min,future_3state_movement_360min,future_middle_return_480min,future_2state_movement_480min,future_3state_movement_480min,future_middle_return_600min,future_2state_movement_600min,future_3state_movement_600min
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,0.0,-0.000675,-1.0,0.0,0.002238,1.0,0.0,0.008904,1.0,1.0
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,0.0,-0.001127,-1.0,0.0,0.001797,1.0,0.0,0.008078,1.0,1.0
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,-1.0,-0.002048,-1.0,0.0,0.000874,1.0,0.0,0.006705,1.0,1.0
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,-1.0,-0.002548,-1.0,0.0,0.000347,1.0,0.0,0.006084,1.0,1.0
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,-1.0,-0.002116,-1.0,0.0,0.000881,1.0,0.0,0.006532,1.0,1.0


In [17]:
top10_1min_df = pd.read_csv(
    f"../data/1min/top10_2019_train_test.csv.gz",
    sep=',',
    parse_dates=["time"],
    index_col=['time', 'pair'],
    infer_datetime_format=True,
    compression='gzip',
)
top10_1min_df.head()

,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,future_3state_movement_240min,future_middle_return_360min,future_2state_movement_360min,future_3state_movement_360min,future_middle_return_480min,future_2state_movement_480min,future_3state_movement_480min,future_middle_return_600min,future_2state_movement_600min,future_3state_movement_600min
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,0.0,-0.000675,-1.0,0.0,0.002238,1.0,0.0,0.008904,1.0,1.0
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,0.0,-0.001127,-1.0,0.0,0.001797,1.0,0.0,0.008078,1.0,1.0
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,-1.0,-0.002048,-1.0,0.0,0.000874,1.0,0.0,0.006705,1.0,1.0
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,-1.0,-0.002548,-1.0,0.0,0.000347,1.0,0.0,0.006084,1.0,1.0
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,-1.0,-0.002116,-1.0,0.0,0.000881,1.0,0.0,0.006532,1.0,1.0


In [16]:
top10_1min_original_df = pd.read_csv(
    f"../data/1min/top10_merged.csv.gz",
    sep=',',
    parse_dates=["time"],
    index_col=['time', 'pair'],
    infer_datetime_format=True,
    compression='gzip',
    usecols=["time", "pair", "middle_median"]
)
top10_1min_original_df.head()

,,middle_median
time,pair,
2013-04-01 02:07:00,btcusd,93.275
2013-04-01 02:08:00,btcusd,100.000
2013-04-01 02:09:00,btcusd,93.300
2013-04-01 02:10:00,btcusd,93.300
2013-04-01 02:11:00,btcusd,93.410


In [18]:
top10_1min_df.shape

(5255990, 59)

In [19]:
top10_1min_df = pd.merge(top10_1min_df, top10_1min_original_df, left_index=True, right_index=True, how="left")
print(top10_1min_df.shape)
top10_1min_df.head()

(5255990, 60)


,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,future_middle_return_360min,future_2state_movement_360min,future_3state_movement_360min,future_middle_return_480min,future_2state_movement_480min,future_3state_movement_480min,future_middle_return_600min,future_2state_movement_600min,future_3state_movement_600min,middle_median
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,-0.000675,-1.0,0.0,0.002238,1.0,0.0,0.008904,1.0,1.0,3826.382003
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,-0.001127,-1.0,0.0,0.001797,1.0,0.0,0.008078,1.0,1.0,3827.582003
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,-0.002048,-1.0,0.0,0.000874,1.0,0.0,0.006705,1.0,1.0,3829.317539
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,-0.002548,-1.0,0.0,0.000347,1.0,0.0,0.006084,1.0,1.0,3832.850000
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,-0.002116,-1.0,0.0,0.000881,1.0,0.0,0.006532,1.0,1.0,3834.769429


In [23]:
# # save to gzip-csv
# (
#     top10_1min_df
#     [ (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]
#     .to_csv(
#         "../data/1min/top10_2019_train_test.csv.gz",
#          sep=",",
#          chunksize=1000000,
#          compression='gzip',
#     )
    
# )

## Add future realized return unter constraint

In [6]:
top10_1min_original_df = pd.read_csv(
    f"../data/1min/top10_2019_train_test.csv.gz",
    sep=',',
    parse_dates=["time"],
    index_col=['time', 'pair'],
    infer_datetime_format=True,
    compression='gzip',
)

In [7]:
top10_1min_original_df.head()

,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,future_middle_return_360min,future_2state_movement_360min,future_3state_movement_360min,future_middle_return_480min,future_2state_movement_480min,future_3state_movement_480min,future_middle_return_600min,future_2state_movement_600min,future_3state_movement_600min,middle_median
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,-0.000675,-1.0,0.0,0.002238,1.0,0.0,0.008904,1.0,1.0,3826.382003
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,-0.001127,-1.0,0.0,0.001797,1.0,0.0,0.008078,1.0,1.0,3827.582003
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,-0.002048,-1.0,0.0,0.000874,1.0,0.0,0.006705,1.0,1.0,3829.317539
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,-0.002548,-1.0,0.0,0.000347,1.0,0.0,0.006084,1.0,1.0,3832.850000
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,-0.002116,-1.0,0.0,0.000881,1.0,0.0,0.006532,1.0,1.0,3834.769429


In [8]:
top10_1min_original_df.index.get_level_values("pair").unique()

Index(['btcusd', 'ethusd', 'eosusd', 'ltcusd', 'xrpusd', 'babusd', 'xmrusd',
       'neousd', 'iotusd', 'dshusd'],
      dtype='object', name='pair')

In [10]:
print(TOP_10_CAPITALIZATION)

['btcusd', 'ethusd', 'eosusd', 'ltcusd', 'xrpusd', 'babusd', 'xmrusd', 'neousd', 'iotusd', 'dshusd']


In [12]:
pairs = top10_1min_original_df.index.get_level_values("pair").unique()
return_list = []
for delta in [120]:
#               240]:
    for pair in pairs:
        top10_1min_original_df[f"future_return_{delta}min_constraint"] = 0
        print("Delta:", delta, "Pair:", pair)
        # take only row from one pair
        pair_df = top10_1min_original_df[ top10_1min_original_df.index.get_level_values("pair") == pair ]
        
        max_row_num = pair_df.shape[0]
        # loop through rows
        for row_num, _ in enumerate(pair_df.iterrows()):
            # check if look forward would yield data, else add NaN to list
            if row_num + delta + 1 < max_row_num:
                # take next row
                _row_num = row_num + 1
                row_1d = pair_df.iloc[ _row_num ]
                # take row in delta bins ahead
                _row_num += delta
                row_future = pair_df.iloc[ _row_num ]

                future_return = np.NaN
                # loop until one reaches bin with any volume for the respective pair
                while _row_num + 1 < max_row_num and np.isnan(future_return):
                    volume = row_future["volume_scaled"]
                    # if not volume go to next bin
                    if not volume:
                        _row_num += 1
                        row_future = pair_df.iloc[ _row_num ]
                    else:
                        # save return
                        future_return = (row_future["middle_median"] - row_1d["middle_median"]) / row_1d["middle_median"]
                return_list.append( future_return )
            else:
                return_list.append( np.NaN )
        # save returns in new colums
#         top10_1min_original_df.loc[ top10_1min_original_df.index.get_level_values("pair") == pair,  f"future_return_{delta}min_constraint" ] = return_list

Delta: 120 Pair: btcusd
Delta: 120 Pair: ethusd
Delta: 120 Pair: eosusd
Delta: 120 Pair: ltcusd
Delta: 120 Pair: xrpusd
Delta: 120 Pair: babusd
Delta: 120 Pair: xmrusd
Delta: 120 Pair: neousd
Delta: 120 Pair: iotusd
Delta: 120 Pair: dshusd


In [27]:
top10_1min_original_df[ top10_1min_original_df.index.get_level_values("pair") == pair ][ ["volume_scaled", "middle_median", "future_return_120min_constraint"] ].tail(127)

,,volume_scaled,middle_median,future_return_120min_constraint
time,pair,,,
2019-12-31 21:53:00,dshusd,0.000000,41.1440,0.002382
2019-12-31 21:54:00,dshusd,0.000000,41.1440,0.003293
2019-12-31 21:55:00,dshusd,0.000000,41.1440,0.003293
2019-12-31 21:56:00,dshusd,0.000000,41.1440,NaN
2019-12-31 21:57:00,dshusd,0.000000,41.1440,NaN
...,...,...,...,...
2019-12-31 23:55:00,dshusd,0.000000,41.2420,NaN
2019-12-31 23:56:00,dshusd,0.886988,41.2795,NaN
2019-12-31 23:57:00,dshusd,0.000000,41.2795,NaN


In [13]:
top10_1min_original_df[ "future_return_120min_constraint" ] = return_list 

In [14]:
future_returns_df = ( 
    top10_1min_original_df
    ["future_return_120min_constraint"]
    .groupby( ["time", "pair"] )
    .first()
    .unstack() 
)
future_returns_df.head()

pair,babusd,btcusd,dshusd,eosusd,ethusd,iotusd,ltcusd,neousd,xmrusd,xrpusd
time,,,,,,,,,,
2019-01-01 00:01:00,0.014117,0.002121,0.004560,-0.006375,-0.000587,-0.000111,0.006318,0.008312,-0.001041,0.000388
2019-01-01 00:02:00,0.014085,0.001275,0.002124,-0.008983,-0.002634,0.001904,0.005087,0.006566,-0.002487,-0.002598
2019-01-01 00:03:00,0.012737,0.000300,0.002124,-0.005316,-0.003045,0.000944,0.004371,0.004634,-0.002487,-0.002915
2019-01-01 00:04:00,0.012409,-0.000188,-0.002592,-0.003753,-0.003397,-0.001592,0.004598,0.004848,-0.005235,-0.002748
2019-01-01 00:05:00,0.010477,0.000375,-0.003780,-0.001925,-0.001720,-0.001592,0.003899,0.004219,-0.005573,-0.002279


In [17]:
future_returns_df["cross_sectional_median_120min"] = future_returns_df.median(axis=1)
future_returns_df["cross_sectional_median_120min"].head()

c:\users\rared\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


time
2019-01-01 00:01:00    0.001254
2019-01-01 00:02:00    0.001589
2019-01-01 00:03:00    0.000622
2019-01-01 00:04:00   -0.002092
2019-01-01 00:05:00   -0.001656
Name: cross_sectional_median_120min, dtype: float64

In [20]:
print(top10_1min_original_df.shape)
top10_1min_original_df = pd.merge( top10_1min_original_df, future_returns_df[ "cross_sectional_median_120min"], left_index=True, right_index=True, how="left" )
print(top10_1min_original_df.shape)

(5255990, 61)
(5255990, 62)


In [39]:
top10_1min_original_df.head()

,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,future_3state_movement_360min,future_middle_return_480min,future_2state_movement_480min,future_3state_movement_480min,future_middle_return_600min,future_2state_movement_600min,future_3state_movement_600min,middle_median,future_return_120min_constraint,cross_sectional_median_120min
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,0.0,0.002238,1.0,0.0,0.008904,1.0,1.0,3826.382003,0.002121,0.001254
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,0.0,0.001797,1.0,0.0,0.008078,1.0,1.0,3827.582003,0.001275,0.001589
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,0.0,0.000874,1.0,0.0,0.006705,1.0,1.0,3829.317539,0.000300,0.000622
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,0.0,0.000347,1.0,0.0,0.006084,1.0,1.0,3832.850000,-0.000188,-0.002092
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,0.0,0.000881,1.0,0.0,0.006532,1.0,1.0,3834.769429,0.000375,-0.001656


In [79]:
def get_3state_price_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    
    if abs(price_return) - transaction_cost > 0:
        if cross_sectional_median_column:
            if price_return > max(row[cross_sectional_median_column], 0):
                return 1
            elif price_return < min(row[cross_sectional_median_column], 0):
                return -1
            else:
                return 0
        else:
            if price_return > 0:
                return 1
            else:
                return -1
    else:
        return 0

def get_2state_up_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    if cross_sectional_median_column:
        if price_return > row[cross_sectional_median_column] and price_return - transaction_cost > 0:
            return 1
        else:
            return 0
    else:
        if price_return - transaction_cost > 0:
            return 1
        else:
            return 0
        
def get_2state_movement(row, return_column, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    else:
        if cross_sectional_median_column:
            if price_return > row[cross_sectional_median_column]:
                return 1
            else:
                return -1
        else:
            if price_return > 0:
                return 1
            else:
                return -1
        
def get_2state_down_movement(row, return_column, transaction_cost=0.003, cross_sectional_median_column=""):
    price_return = row[ return_column ]
    if pd.isnull(price_return):
        return price_return
    if cross_sectional_median_column:
        if price_return < row[cross_sectional_median_column] and -price_return - transaction_cost > 0:
            return -1
        else:
            return 0
    else:
        if -price_return - transaction_cost > 0:
            return -1
        else:
            return 0           

In [82]:
print("Get 3state movement")
top10_1min_original_df["future_3state_movement_120min_30bps"] = top10_1min_original_df.apply(get_3state_price_movement, return_column="future_return_120min_constraint", transaction_cost=0.003, cross_sectional_median_column="cross_sectional_median_120min", axis=1)
print("Get 2state up-movement")
top10_1min_original_df["future_2state_up_movement_120min_30bps"] = top10_1min_original_df.apply(get_2state_up_movement, return_column="future_return_120min_constraint", transaction_cost=0.003, cross_sectional_median_column="cross_sectional_median_120min", axis=1)
print("Get 2state down-movement")
top10_1min_original_df["future_2state_down_movement_120min_30bps"] = top10_1min_original_df.apply(get_2state_down_movement, return_column="future_return_120min_constraint", transaction_cost=0.003, cross_sectional_median_column="cross_sectional_median_120min", axis=1)

print("Get 2state movement")
top10_1min_original_df["future_2state_movement_120min"] = top10_1min_original_df.apply(get_2state_movement, return_column="future_return_120min_constraint", cross_sectional_median_column="cross_sectional_median_120min", axis=1)

Get 3state movement
Get 2state up-movement
Get 2state down-movement
Get 2state movement


In [83]:
top10_1min_original_df.head()

,,middle_return_1min,volume_scaled,middle_return_5min,volume_scaled_5min,middle_return_10min,volume_scaled_10min,middle_return_20min,volume_scaled_20min,middle_return_40min,volume_scaled_40min,...,volume_scaled_1680min,middle_return_1920min,volume_scaled_1920min,middle_median,future_return_120min_constraint,cross_sectional_median_120min,future_3state_movement_120min_30bps,future_2state_up_movement_120min_30bps,future_2state_down_movement_120min_30bps,future_2state_movement_120min
time,pair,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:01:00,btcusd,-0.000378,0.148808,0.000823,3823.234516,0.000780,3823.400000,0.001416,3820.972565,0.004999,3807.35,...,3940.950000,-0.029168,3941.343073,3826.382003,0.002121,0.001254,0.0,0.0,0.0,1.0
2019-01-01 00:02:00,btcusd,0.000314,0.971417,0.000884,3824.200000,0.001434,3822.100000,0.001574,3821.565333,0.005248,3807.60,...,3942.922533,-0.028964,3941.750000,3827.582003,0.001275,0.001589,0.0,0.0,0.0,-1.0
2019-01-01 00:03:00,btcusd,0.000453,0.854522,0.001325,3824.250000,0.001866,3822.184516,0.002029,3821.565333,0.005638,3807.85,...,3942.922533,-0.029471,3945.600000,3829.317539,0.000300,0.000622,0.0,0.0,0.0,-1.0
2019-01-01 00:04:00,btcusd,0.000922,0.943426,0.001542,3826.950000,0.002786,3822.200000,0.002857,3821.930113,0.006526,3808.00,...,3943.000000,-0.030628,3953.950000,3832.850000,-0.000188,-0.002092,0.0,0.0,0.0,1.0
2019-01-01 00:05:00,btcusd,0.000501,0.422035,0.001814,3827.827600,0.003293,3822.184516,0.002900,3823.681246,0.007043,3807.95,...,3944.800000,-0.030730,3956.350000,3834.769429,0.000375,-0.001656,0.0,0.0,0.0,1.0


In [87]:
top10_1min_original_df[ top10_1min_original_df["volume_scaled"].shift(-1) != 0 ]["future_3state_movement_120min_30bps"].value_counts(normalize=True)

 0.0    0.522300
 1.0    0.240113
-1.0    0.237587
Name: future_3state_movement_120min_30bps, dtype: float64

In [88]:
top10_1min_original_df[ top10_1min_original_df["volume_scaled"].shift(-1) != 0 ]["future_2state_up_movement_120min_30bps"].value_counts(normalize=True)

0.0    0.759887
1.0    0.240113
Name: future_2state_up_movement_120min_30bps, dtype: float64

In [89]:
top10_1min_original_df[ top10_1min_original_df["volume_scaled"].shift(-1) != 0 ]["future_2state_down_movement_120min_30bps"].value_counts(normalize=True)

 0.0    0.762413
-1.0    0.237587
Name: future_2state_down_movement_120min_30bps, dtype: float64

In [90]:
top10_1min_original_df[ top10_1min_original_df["volume_scaled"].shift(-1) != 0 ]["future_2state_movement_120min"].value_counts(normalize=True)

 1.0    0.501762
-1.0    0.498238
Name: future_2state_movement_120min, dtype: float64

In [84]:
# save to gzip-csv
(
    top10_1min_original_df
    [ (top10_1min_original_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_original_df.index.get_level_values("time") < "2020-01-01") ]
    .to_csv(
        "../data/1min/top10_2019_train_test_new.csv.gz",
         sep=",",
         chunksize=1000000,
         compression='gzip',
    )  
)

## Investigate movements

In [ ]:
movement_2state = (
    top10_1min_df
    [ [x for x in top10_1min_df.columns if "2state" in x ] ]
    [ (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]
    .melt(var_name='intervall', value_name='movement')
)

movement_2state["movement"] = movement_2state["movement"].map({ -1.0: "down", 0: "stable", 1.0: "up"})

movement_2state["count"] = 1

movement_2state = movement_2state.groupby(["intervall", "movement"]).agg(class_count=("count", "count")).reset_index()

In [ ]:
fig = px.bar(movement_2state, x="intervall", y="class_count", color="movement", barmode="group", title="Distribution of movements in 2019 per return-intervall")
fig.show()
pio.write_html(fig, file='exploration/1min/movement_2state_distribution.html')

In [ ]:
movement_2state_only_volume = (
    top10_1min_df
    [ [x for x in top10_1min_df.columns if "2state" in x ] ]
    [ (top10_1min_df["volume_scaled"] != 0) & (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]
    .melt(var_name='intervall', value_name='movement')
)

movement_2state_only_volume["movement"] = movement_2state_only_volume["movement"].map({ -1.0: "down", 0: "stable", 1.0: "up"})

movement_2state_only_volume["count"] = 1

movement_2state_only_volume = movement_2state_only_volume.groupby(["intervall", "movement"]).agg(class_count=("count", "count")).reset_index()

In [ ]:
fig = px.bar(movement_2state_only_volume, x="intervall", y="class_count", color="movement", barmode="group", title="Distribution of movements in 2019 per return-intervall")
fig.show()
pio.write_html(fig, file='exploration/1min/movement_2state_only_volume_distribution.html')

In [ ]:
movement_3state = (
    top10_1min_df
    [ [x for x in top10_1min_df.columns if "3state" in x ] ]
    [ (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]
    .melt(var_name='intervall', value_name='movement'))

movement_3state["movement"] = movement_3state["movement"].map({ -1.0: "down", 0: "stable", 1.0: "up"})

movement_3state["count"] = 1

movement_3state = movement_3state.groupby(["intervall", "movement"]).agg(class_count=("count", "count")).reset_index()

In [ ]:
fig = px.bar(
    movement_3state, 
    x="intervall", 
    y="class_count", 
    color="movement", 
    title="Distribution of movements in 2019 per return-intervall"
#     barmode="group"
)
fig.show()
pio.write_html(fig, file='exploration/1min/movement_3state_distribution.html')

In [ ]:
movement_3state_only_volume = (
    top10_1min_df
    [ [x for x in top10_1min_df.columns if "3state" in x ] ]
    [ (top10_1min_df["volume_scaled"] != 0) & (top10_1min_df.index.get_level_values("time") > "2019-01-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]

    .melt(var_name='intervall', value_name='movement')
)

movement_3state_only_volume["movement"] = movement_3state_only_volume["movement"].map({ -1.0: "down", 0: "stable", 1.0: "up"})

movement_3state_only_volume["count"] = 1

movement_3state_only_volume = movement_3state_only_volume.groupby(["intervall", "movement"]).agg(class_count=("count", "count")).reset_index()

fig = px.bar(
    movement_3state_only_volume, 
    x="intervall", 
    y="class_count", 
    color="movement", 
    title="Distribution of movements in 2019 per return-intervall only considering bins with trades"
#     barmode="group"
)
fig.show()
pio.write_html(fig, file='exploration/1min/movement_3state_only_volume_distribution.html')

In [ ]:
# top10_1min_df = pd.read_csv(
#     "../data/1min/top10_merged.csv.gz",
#     sep=',',
#     parse_dates=["time"],
#     index_col=['time', 'pair'],
#     infer_datetime_format=True,
#     compression='gzip',
# )

In [ ]:
# top10_1min_df = top10_1min_df[ (top10_1min_df.index.get_level_values("time") > "2019-11-01") & (top10_1min_df.index.get_level_values("time") < "2020-01-01") ]

In [ ]:
# top10_1min_df.head()